In [ ]:
# 02_strategy_prototyping.ipynb

# Import libraries
import pandas as pd
import matplotlib.pyplot as plt

# --- Load Data ---
symbol = 'AAPL'
data_path = f'../data/{symbol}.csv'
try:
    df = pd.read_csv(data_path, index_col='Date', parse_dates=True)
except FileNotFoundError:
    print(f"Error: Data file not found at {data_path}.")
    df = None

if df is not None:
    # --- Simple Moving Average Strategy Prototype ---
    short_window = 20
    long_window = 50

    df['short_ma'] = df['Close'].rolling(window=short_window, min_periods=1).mean()
    df['long_ma'] = df['Close'].rolling(window=long_window, min_periods=1).mean()
    df['signals'] = 0  # 0: Hold, 1: Buy, -1: Sell

    # Generate buy signals
    df['signals'][(df['short_ma'].shift(1) < df['long_ma'].shift(1)) & (df['short_ma'] > df['long_ma'])] = 1

    # Generate sell signals
    df['signals'][(df['short_ma'].shift(1) > df['long_ma'].shift(1)) & (df['short_ma'] < df['long_ma'])] = -1

    # --- Visualize Signals ---
    plt.figure(figsize=(14, 7))
    plt.plot(df['Close'], label='Close Price')
    plt.plot(df['short_ma'], label=f'{short_window}-day MA', alpha=0.7)
    plt.plot(df['long_ma'], label=f'{long_window}-day MA', alpha=0.7)
    plt.scatter(df.loc[df['signals'] == 1].index, df['Close'][df['signals'] == 1], marker='^', color='g', label='Buy Signal')
    plt.scatter(df.loc[df['signals'] == -1].index, df['Close'][df['signals'] == -1], marker='v', color='r', label='Sell Signal')
    plt.title(f'{symbol} with Moving Average Crossover Signals')
    plt.xlabel('Date')
    plt.ylabel('Price')
    plt.legend()
    plt.grid(True)
    plt.show()

    # --- Simple Backtesting (Illustrative) ---
    df['positions'] = df['signals'].shift(1).fillna(0)
    df['returns'] = df['Close'].pct_change().fillna(0)
    df['strategy_returns'] = df['positions'] * df['returns']
    cumulative_returns = (1 + df['strategy_returns']).cumprod()

    plt.figure(figsize=(14, 7))
    plt.plot(cumulative_returns, label='Strategy Cumulative Returns')
    plt.title('Simple Strategy Cumulative Returns')
    plt.xlabel('Date')
    plt.ylabel('Cumulative Returns')
    plt.legend()
    plt.grid(True)
    plt.show()

    # --- Further Experimentation ---
    # You can now try different MA windows, other indicators, etc.